# Confined transient (small streams) – base model

Set up the confined transient MODFLOW 6 model using the locally prepared inputs under `mountain/`. This mirrors the earlier `Confined_version_transient_smallstreams` workflow but pulls rasters/NPZ from the new folders so it can run self-contained.

In [14]:
import os
from pathlib import Path
import math
import numpy as np
import pandas as pd
import geopandas as gpd
import shapefile as sf
from shapely.geometry import Polygon
import shutil

import flopy
from flopy.discretization import StructuredGrid
from flopy.utils.gridintersect import GridIntersect

print('flopy version:', flopy.__version__)

# Paths
mountain_dir = Path('/home/jec18/modflow/jupy/VUB/mountain')
reproj_dir = mountain_dir / 'reprojected'
reproj_rasters = reproj_dir / 'rasters'
reproj_vectors = reproj_dir / 'vectors'
prepared_dir = mountain_dir / 'prepared'

pre_npz = prepared_dir / 'preprocessed_confined_inputs.npz'
if not pre_npz.exists():
    raise FileNotFoundError(f"Missing preprocessed NPZ: {pre_npz}")

catchment_path = Path('/home/jec18/catchments/km105.shp')
# Use rivers from the clipped/reprojected prep (HydroRIVERS) instead of the old psad shapefile
rivers_path = reproj_vectors / 'rivers_hydrorivers_psad.gpkg'
if not rivers_path.exists():
    rivers_path = Path('/home/jec18/datasets/gw/psad/rivers_valid/rivers_sfr.shp')
workspace = mountain_dir / 'base_model'
# clean workspace each run
if workspace.exists():
    shutil.rmtree(workspace)
workspace.mkdir(exist_ok=True)
name = 'vub'


flopy version: 3.9.5


## Grid and inputs from NPZ
Use the prepared NPZ to rebuild the StructuredGrid, arrays, and recharge/ET stacks.

In [15]:
data = np.load(pre_npz)
resolution = int(data['resolution'])
xmin = float(data['xmin'])
ymin = float(data['ymin'])
nrow = int(data['nrow'])
ncol = int(data['ncol'])

# Rebuild grid
delr = resolution * np.ones(ncol, dtype=float)
delc = resolution * np.ones(nrow, dtype=float)
top = np.full((nrow, ncol), 1.0)
botm = np.full((1, nrow, ncol), 0.0)

struct_grid = StructuredGrid(
    nlay=1,
    delr=delr,
    delc=delc,
    xoff=xmin,
    yoff=ymin,
    top=top,
    botm=botm,
)
struct_grid.idomain = data['idomain'].reshape(struct_grid.shape)

# Arrays
pre_topo = data['topo']
k_h_data = data['k_h_data']
k_h_data2 = data['k_h_data2']
recharge_data = data['recharge_data']
deposits = data['deposits']
bedrock = data['bedrock']

print('Grid shape:', struct_grid.shape)


Grid shape: (1, 608, 678)


In [16]:
# Load rivers for potential RIV/DRN setup
rivers_gdf = gpd.read_file(rivers_path)
if rivers_gdf.crs is None:
    rivers_gdf = rivers_gdf.set_crs('EPSG:4326')
rivers_gdf = rivers_gdf.to_crs(epsg=24893)
print('Rivers loaded:', rivers_path)
print('Rivers CRS:', rivers_gdf.crs)


Rivers loaded: /home/jec18/modflow/jupy/VUB/mountain/reprojected/vectors/rivers_hydrorivers_psad.gpkg
Rivers CRS: EPSG:24893


## Time discretization
Steady-state spinup + transient monthly periods (10 years). Adjust as needed.

In [17]:
spinup_years = 10
months_per_year = 12
nper_tr = spinup_years * months_per_year
nper_total = 1 + nper_tr

days_per_month = 30.44
seconds_per_day = 86400
perlen = [1.0] + [days_per_month * seconds_per_day] * nper_tr
nstp   = [1] + [3] * nper_tr
tsmult = [1.0] * nper_total
sp_type = ['SS'] + ['TR'] * nper_tr


## Build MODFLOW 6 simulation
Sets up DIS, IC, NPF, STO, RCH with monthly files. River/Drain packages can be added using the same pattern as the prior notebook (see placeholder section).

In [18]:
# Prepare recharge text files in workspace
workspace_path = Path(workspace)
monthly_rch = recharge_data[1:]
month_files = []
for month_idx, arr in enumerate(monthly_rch, start=1):
    fname = workspace_path / f"recharge_month_{month_idx:02d}.txt"
    np.savetxt(fname, arr)
    month_files.append(fname.name)
ss_file = workspace_path / "recharge_steady.txt"
np.savetxt(ss_file, recharge_data[0])

# Build simulation
sim = flopy.mf6.MFSimulation(sim_ws=str(workspace_path), exe_name='mf6', verbosity_level=2)
tdis = flopy.mf6.ModflowTdis(sim, time_units='seconds', nper=nper_total, perioddata=list(zip(perlen, nstp, tsmult)))
gwf = flopy.mf6.ModflowGwf(sim, modelname=name, save_flows=True, print_input=True)
ims = flopy.mf6.ModflowIms(sim, complexity='simple', print_option='SUMMARY', linear_acceleration='bicgstab')

dis = flopy.mf6.ModflowGwfdis(
    gwf,
    nlay=1,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=1,
    botm=0,
    xorigin=xmin,
    yorigin=ymin,
    idomain=struct_grid.idomain,
    length_units='METERS'
)

ic = flopy.mf6.ModflowGwfic(gwf, strt=pre_topo)

# Hydraulic conductivity
k = np.zeros((1, nrow, ncol))
k[0, :, :] = (k_h_data * 40 + k_h_data2 * 100) / 140
k = np.nan_to_num(k, nan=1e-12)

npf = flopy.mf6.ModflowGwfnpf(gwf, icelltype=0, k=k, save_specific_discharge=True)

sto = flopy.mf6.ModflowGwfsto(
    gwf,
    save_flows=True,
    ss=3e-2,
    steady_state={0: True},
    transient={1: True}
)

# Recharge package

def openclose(name):
    return ['open/close', name, 'factor', 1.0]

rch_spd = {0: openclose(ss_file.name)}
for kper in range(1, nper_total):
    month_idx = (kper - 1) % len(month_files)
    rch_spd[kper] = openclose(month_files[month_idx])

rch = flopy.mf6.ModflowGwfrcha(gwf, print_flows=True, recharge=rch_spd)

# Output control
budget_file = f"{name}.bud"
head_file = f"{name}.hds"
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    budget_filerecord=budget_file,
    head_filerecord=head_file,
    saverecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')]
)


## Rivers and drains
Generate RIV/DRN packages from the clipped rivers, separating major (> Strahler 2) and minor (<=2) streams, and write stress-period data to external template files.

## Run simulation

In [19]:
from shapely.geometry import LineString, MultiLineString

# Build river multilines (split by Strahler order)
order_col = 'ORD_STRA' if 'ORD_STRA' in rivers_gdf.columns else None
if order_col is None:
    for cand in ['ord_stra', 'strahler', 'STRAHLER']:
        if cand in rivers_gdf.columns:
            order_col = cand
            break
if order_col is None:
    raise ValueError('Could not find Strahler order column in rivers_gdf')

riverlist = []
smallstream_list = []
order_threshold = 2
for geom, order in zip(rivers_gdf.geometry, rivers_gdf[order_col]):
    if geom is None or geom.is_empty:
        continue
    parts = list(geom.geoms) if geom.geom_type == 'MultiLineString' else [geom]
    order_val = float(order)
    target_list = riverlist if order_val > order_threshold else smallstream_list
    for part in parts:
        if part.is_empty:
            continue
        target_list.append(LineString(part))

riv = MultiLineString(riverlist) if riverlist else None
minor_riv = MultiLineString(smallstream_list) if smallstream_list else None

ix = GridIntersect(struct_grid)
ixs = ix.intersect(riv) if riv is not None else []
ixs_minor = ix.intersect(minor_riv) if minor_riv is not None else []

# River stress-period data
active_idomain = struct_grid.idomain[0]
riv_data = []
for inter in ixs:
    if active_idomain[inter.cellids] > 0:
        riverbed = pre_topo[inter.cellids]
        cond = 1e-4 * inter.lengths * 30.0
        stage = riverbed - 5.0
        riv_data.append(((0, *inter.cellids), riverbed, cond, stage))

# Drains for minor streams
drains_width = 2.0
drn_data = []
minor_cells = set()
for inter in ixs_minor:
    if active_idomain[inter.cellids] > 0:
        stage = pre_topo[inter.cellids] - 1.0
        cond = 1e-5 * inter.lengths * drains_width
        drn_data.append(((0, *inter.cellids), stage, cond))
        minor_cells.add(tuple(inter.cellids))

# Diffuse drains elsewhere
background_cond = 1e-10 * (resolution ** 2)
drn_dif_data = []
active_indices = np.argwhere(active_idomain > 0)
for row, col in active_indices:
    if (row, col) in minor_cells:
        continue
    stage = pre_topo[row, col]
    drn_dif_data.append(((0, row, col), stage, background_cond))

# Write to external files
def write_list_records(path, records):
    with path.open("w") as f:
        for (lay, row, col), *vals in records:
            lay += 1
            row += 1
            col += 1
            f.write(f"{lay:3d} {row:3d} {col:3d} " + " ".join(f"{v:.8g}" for v in vals) + "\n")
riv_file = workspace_path / "riv_template.dat"
write_list_records(riv_file, riv_data)

riv_spd = {k: {"filename": riv_file.name} for k in range(nper_total)}

mfriv = flopy.mf6.ModflowGwfriv(
    gwf,
    stress_period_data=riv_spd,
    save_flows=True,
    filename="model.riv",
)

drn_file = workspace_path / "drn_template.dat"
write_list_records(drn_file, drn_data)

drn_spd = {k: {"filename": drn_file.name} for k in range(nper_total)}

drn = flopy.mf6.ModflowGwfdrn(
    gwf,
    stress_period_data=drn_spd,
    save_flows=True,
    filename="model.drn",
    pname="DRN-riv"
)

drn_dif_file = workspace_path / "drn_dif_template.dat"
write_list_records(drn_dif_file, drn_dif_data)

drn_dif_spd = {k: {"filename": drn_dif_file.name} for k in range(nper_total)}

drn_dif = flopy.mf6.ModflowGwfdrn(
    gwf,
    stress_period_data=drn_dif_spd,
    save_flows=True,
    filename="model_dif.drn",
    pname="DRN-dif"
)
print('RIV/DRN packages added with external template files')


/home/jec18/hydro1/anaconda3/envs/gmdsitut/lib/python3.11/site-packages/flopy/utils/gridintersect.py:123: DeprecationWarning: Note `method="structured"` is deprecated. Pass `method="vertex"` to silence this warning. This will be the new default in a future release and this keyword argument will be removed.
  warnings.warn(


RIV/DRN packages added with external template files


In [20]:
sim.write_simulation()
success, buff = sim.run_simulation(report=True)
print('Simulation success:', success)


writing simulation...
  writing simulation name file...
      writing block options...
        writing data continue...
        writing data nocheck...
        writing data memory_print_option...
        writing data profile_option...
        writing data maxerrors...
        writing data print_input...
        writing data hpc_filerecord...
      writing block timing...
        writing data tdis6...
      writing block models...
        writing data models...
      writing block exchanges...
        writing data exchanges...
      writing block solutiongroup...
        writing data mxiter (0)...
        writing data solutiongroup (0)...
  writing simulation tdis package...
      writing block options...
        writing data time_units...
        writing data start_date_time...
        writing data ats_filerecord...
      writing block dimensions...
        writing data nper...
      writing block perioddata...
        writing data perioddata...
  writing solution package ims_-1...
   

## Postprocessing hooks
Add head/budget processing, diagnostics, and catchment discharge aggregation as needed (reuse functions from the earlier notebook).